# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [37]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [38]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jamestown,42.0970,-79.2353,276.22,37,0,4.12,US,1680455750
1,1,abha,18.2164,42.5053,290.05,77,20,2.06,SA,1680455931
2,2,yanji,42.9075,129.5078,279.53,83,22,1.36,CN,1680455931
3,3,santa cruz,-17.8000,-63.1667,304.85,58,40,1.54,BO,1680455932
4,4,tiksi,71.6872,128.8694,263.07,78,100,14.57,RU,1680455933


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [39]:
%%capture --no-display

# Configure the map plot
scatter_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    size='Humidity',
    color='blue',
    alpha=0.5,
    height=500,
    xlim=(-180, 180),
    ylim=(-90, 90),
    hover_cols=['City'],
)

# Display the map
scatter_plot

:Scatter   [Lng]   (Lat,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [40]:
# Narrow down cities that fit criteria and create a new DataFrame
ideal_cities_df = city_data_df.loc[(city_data_df['Max Temp'] > 20)].copy()

# Drop any rows with null values
ideal_cities_df.dropna(inplace=True)

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jamestown,42.0970,-79.2353,276.22,37,0,4.12,US,1680455750
1,1,abha,18.2164,42.5053,290.05,77,20,2.06,SA,1680455931
2,2,yanji,42.9075,129.5078,279.53,83,22,1.36,CN,1680455931
3,3,santa cruz,-17.8000,-63.1667,304.85,58,40,1.54,BO,1680455932
4,4,tiksi,71.6872,128.8694,263.07,78,100,14.57,RU,1680455933


### Step 3: Create a new DataFrame called `hotel_df`.

In [41]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.loc[:, ['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,jamestown,US,42.0970,-79.2353,37,
1,abha,SA,18.2164,42.5053,77,
2,yanji,CN,42.9075,129.5078,83,
3,santa cruz,BO,-17.8000,-63.1667,58,
4,tiksi,RU,71.6872,128.8694,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000  # in meters
params = {
    "categories": "accommodation.hotels",
    "limit": 1,
    "radius": radius,
    "sort": "distance",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat}@{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(f"{base_url}?{'&'.join([f'{k}={v}' for k,v in params.items()])}")

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
jamestown - nearest hotel: No hotel found
abha - nearest hotel: No hotel found
yanji - nearest hotel: No hotel found
santa cruz - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
dikson - nearest hotel: No hotel found
tuktoyaktuk - nearest hotel: No hotel found
cravo norte - nearest hotel: No hotel found
sovetskiy - nearest hotel: No hotel found
chuy - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
rikitea - nearest hotel: No hotel found
rabat - nearest hotel: No hotel found
shar - nearest hotel: No hotel found
cervia - nearest hotel: No hotel found
jalu - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
chiredzi - nearest hotel: No hotel found
khatanga - nearest hotel: No hotel found
chokurdakh - nearest hotel: No hotel found
punta arenas - nearest hotel: No hotel found
kalabo - nearest hotel: No hotel found
bluff - nearest hotel: No hotel found
kapaa - neares

saint-georges - nearest hotel: No hotel found
ribeira grande - nearest hotel: No hotel found
anloga - nearest hotel: No hotel found
lethem - nearest hotel: No hotel found
te anau - nearest hotel: No hotel found
naryan-mar - nearest hotel: No hotel found
okhotsk - nearest hotel: No hotel found
fortuna - nearest hotel: No hotel found
brae - nearest hotel: No hotel found
paris - nearest hotel: No hotel found
lincoln - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
micheweni - nearest hotel: No hotel found
rokytnice nad jizerou - nearest hotel: No hotel found
hue - nearest hotel: No hotel found
alyangula - nearest hotel: No hotel found
nanakuli - nearest hotel: No hotel found
breaza - nearest hotel: No hotel found
tuatapere - nearest hotel: No hotel found
wanning - nearest hotel: No hotel found
hilton head island - nearest hotel: No hotel found
xuddur - nearest hotel: No hotel found
san carlos de bariloche - nearest hotel: No hotel found
dingle - nearest hotel

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot with hover information
hover = hvplot.HoverTool(
    tooltips=[
        ("City", "@City"),
        ("Country", "@Country"),
        ("Humidity", "@Humidity"),
        ("Hotel Name", "@{Hotel Name}")
    ]
)

map_plot = hotel_df.hvplot.scatter(
    x="Lng",
    y="Lat",
    c="Humidity",
    cmap="YlGnBu",
    size="Humidity",
    alpha=0.8,
    hover_cols=["City", "Country", "Humidity", "Hotel Name"],
    tools=[hover],
    title="City Map with Nearest Hotels"
)

# Display the map
map_plot